In [ ]:
# === Chargement des librairies ===
import rasterio
from rasterio.plot import show
import numpy as np
import matplotlib.pyplot as plt

# Pour voir toutes les infos dans le terminal
print("Rasterio version :", rasterio.__version__)


In [ ]:
# === Ouvrir un raster ===
path = r"C:\Users\guigu\Documents\PRO\Stage_SERTIT\outil_detection_changement\data\raw\testnotebook\t3\Subset_S1A_IW_GRDH_1SDV_20190702T092435_20190702T092500_027936_032774_949A_Orb_NR_Cal_Spk_TC_GLCM.tif"
path_img2 = r"C:\Users\guigu\Documents\PRO\Stage_SERTIT\outil_detection_changement\data\raw\testnotebook\t3\Subset_S1A_IW_GRDH_1SDV_20190726T092436_20190726T092501_028286_033210_6DDD_Orb_NR_Cal_Spk_TC_GLCM.tif"
# r avant pour raw sinon interprète mal les antislash

with rasterio.open(path) as src:
    print(f"Dimensions : {src.width} x {src.height}") # mieux d'utiliser f-strings (pas comme dans toutes les lignes qui suivent !!!!), plus lisible + rapide 
    print("Nombre de bandes :", src.count)
    print("Système de coordonnées (CRS) :", src.crs)
    print("Résolution :", src.res)  # dans l'unité du CRS du fichier ici mètres (UTM), si image en coordonnées géographique EPSG: 4326 par exemple alors degrés 
    print("Type de données :", src.dtypes)
    print("Bornes géographiques :", src.bounds)
    
    # Lecture de la première bande
    band1 = src.read(1)   # arr = src.read()   shape : (8 (nbr de bande), height, width)
    profile = src.profile # renvoie un dictionnaire contenant toute la configuration du GeoTIFF, les métadonnées techniques du fichier : dimensions, nombre de bandes, type des données, nodata, système de coordonnées, transform affine, etc.


# Affichage de la bande
plt.figure(figsize=(7, 5))
plt.imshow(band1, cmap="gray")
plt.title("Première bande")
plt.colorbar(label="Valeurs de pixel")
plt.show()

print("min:", band1.min(), "max:", band1.max(), "mean:", band1.mean())

fig1, ax1 = plt.subplots(figsize=(7,5))  # subplots indispensable pour plusieurs images dans la même figure, plusierus colorbars ajuster tailles et titres etc
band1 = np.where(band1 == -9999, np.nan, band1) # np.where(condition, valeur_si_vrai, valeur_si_faux) on masque les données à -9999 en les transformant dans le format Nnodata nan qu'on masquera partout par la suite 
img=ax1.imshow(band1, cmap="gray")
ax1.set_title("Bande 1 sans -9999")
fig1.colorbar(img, ax=ax1, label="Valeurs de pixel")

# print("min:", band1.min(), "max:", band1.max(), "mean:", band1.mean())

print("min:", np.nanmin(band1), "max:", np.nanmax(band1), "mean:", np.nanmean(band1)) # version qui ignore les nan 



In [ ]:
# Affichage avec contraste ajusté 

# p1, p99 = np.percentile(band1, (1, 99))
# print("1er percentile:", p1, "99e percentile:", p99)

p1, p99 = np.nanpercentile(band1, (1, 99))   # version qui ignore les nan 
print("1er percentile:", p1, "99e percentile:", p99)


plt.figure()
plt.imshow(band1, cmap='gray', vmin=p1, vmax=p99)
plt.title("Affichage avec contraste ajusté")
plt.colorbar()

# Pour afficher plusieurs figures proprement, plutôt que d'enchaîner les plt.figure 

# fig1, ax1 = plt.subplots(figsize=(7,5))
# img=ax1.imshow(band1, cmap="gray", vmin=p1, vmax=p99)
# fig1.colorbar(img, ax=ax1, label="Valeurs de pixel")
# ax1.set_title("Bande 1")

# fig2, ax2 = plt.subplots(figsize=(7,5))
# ax2.imshow(band2, cmap="gray")
# ax2.set_title("Bande 2")

# plt.show()


In [ ]:
# plt.hist(band1.flatten(), bins=100, color='steelblue') # flatten() transforme ton image 2D (height × width) en un grand vecteur 1D nécessaire pour histogramme
# version robuste plt.hist(band1[np.isfinite(band1)], bins=100, color='steelblue')
# plt.title("Histogramme des valeurs de la bande 1")
# plt.xlabel("Valeur de pixel")
# plt.ylabel("Fréquence")
# plt.show()

fig, ax = plt.subplots(figsize=(7, 5))

# Histogramme robuste (ignorer les NaN si besoin) ax.hist(band1[np.isfinite(band1)], bins=100, color='steelblue')

ax.hist(band1.flatten(), bins=100, color='steelblue')
ax.set_title("Histogramme des valeurs de la bande 1")
ax.set_xlabel("Valeur de pixel")
ax.set_ylabel("Fréquence")

plt.show()



In [ ]:
# === Si ton raster a plusieurs bandes ===
with rasterio.open(path) as src:
    img = src.read()  # shape = (bandes, hauteur, largeur)

print("Dimensions de l'image :", img.shape) # img.shape[0]=nombre de bandes, img.shape[1] = hauteur, img.shape[2] = largeur

# Afficher les 3 premières bandes comme une image RGB
if img.shape[0] >= 3:
    rgb = np.moveaxis(img[:3], 0, -1) 
    # np.moveaxis(arr, 0, -1) prends le canal 0 de mon array et le renvoie à la fin 
    # img[:3] donne un tableau de dimension (3, height, width), moveaxis change la forme de mon parallépipède
    # img[:3].shape = (3, H, W), rgb.shape = (H, W, 3)
    # motivation: Rasterio → (bands, height, width) Matplotlib → (height, width, channels)
    

    plt.figure(figsize=(7,7))
    rgb = np.clip(rgb, 0, None) # np.clip(array, min, max) force toutes les valeurs du tableau : à être ≥ min à être ≤ max (si inferieur à min prends valeur min idem pour max)
    plt.imshow(rgb / np.max(rgb)) # imshow a besoin de valeurs entre 0 et 1 pour afficher des RGB, donc on normalise l'image
    plt.title("Aperçu RGB")
    plt.show()


In [ ]:
# === Création d'une copie avec normalisation simple ===
out_path = r"C:\Users\gbonlieu\Documents\codepythonoutil\outil_detection_changement\ouputs\output_vrac\rasteriotest_output.tiff"

# possibilité de mettre le chemin absolu ou relatif (cf ci-dessous) Ce fichier sera créé dans un dossier output_vr lui même crée dans outputs (existe déjà) / "newdossier/mon.tiff" dans un dossier crée situé dans le répertoire courant (= là où lance le script ou notebook)

# from pathlib import Path

# # === 1. Définir le chemin relatif ===
# out_path = Path("outputs/output_vr/rasteriotest_output.tif")
# out_path.parent.mkdir(parents=True, exist_ok=True)

# Normalisation (entre 0 et 1)
norm = (band1 - np.nanmin(band1)) / (np.nanmax(band1) - np.nanmin(band1))

with rasterio.open(path) as src:
    profile = src.profile  # copie des métadonnées
    profile.update(dtype=rasterio.float32) # Ton image normalisée norm contient des float entre 0 et 1, le raster original peut avoir un type qui ne supporte pas les floats

    with rasterio.open(out_path, "w", **profile) as dst: 
    # profile = {"dtype": float32, "width": 100, "height": 100}, **profile dépaquette en arguments nommés comme si on les avait écrit à la main **profile=dtype=float32, width=100, height=100
        dst.write(norm.astype(rasterio.float32), 1) # convertit le tableau en float 32 bits, dst.write(array, band_index) band_index numero de la band ou on veut écrire 

print("✅ Raster exporté :", out_path)


In [ ]:
with rasterio.open(out_path) as check:
    show(check, cmap="gray")
    print("Nouveau raster ->", check.width, "x", check.height, "| CRS :", check.crs)
